# Prioritized Experience Replay

> Fill in a module description here

In [ ]:
#| default_exp replay

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import torch
from torch import nn
import torch.nn.functional as F
from torch.distributions import Bernoulli, Uniform

from torchtyping import TensorType

### Prioritized Level Replay

In [ ]:
#| export
class PrioritizedReplayDistribution:
    def __init__(
        self,
        staleness_coeff: float = 0.1,
        temperature: float = 0.1, # the beta coefficient for the P_S distribution
    ) -> None:
        self.staleness_coeff = staleness_coeff
        self.temperature = temperature
    
    def create(
        self,
        score_levels: TensorType["n_visited_levels"],
        last_count_levels: TensorType["n_visited_levels"], # the last episode that each level was played,
        last_episode: TensorType[1] # the last episode 
    ) -> TensorType["n_visited_levels"]:
        """Create a prioritized level distribution."""
        
        level_scores = torch.pow(
            input=F.normalize(score_levels, dim=-1),
            exponent=1/self.temperature
        )
        score_dist = level_scores / level_scores.sum(dim=-1)[:, None]
        
        stale_scores = last_episode - last_count_levels
        stale_dist = stale_scores / stale_scores.sum(dim=-1)
        
        prioritized_dist = (1 - self.staleness_coeff) * score_dist + self.staleness_coeff * stale_dist
        
        return prioritized_dist

In [ ]:
#| export
class PrioritizedReplay:
    def __init__(
        self,
        levels: TensorType["n_levels"],
        policy: nn.Module,
    ) -> None:
        self.levels = levels
        # self.visited_levels = torch.zeros_like(levels)
        self.visited_count_levels = torch.zeros_like(self.levels)
        self.last_count_levels = torch.zeros_like(self.levels)
        self.score_levels = torch.zeros_like(levels)
        self.last_episode = 0
        
        self.policy = policy
        self.prioritized_dist = PrioritizedReplayDistribution()
    
    def sample_next_level(
        self,
        visited_levels: TensorType["batch_size", "n_visited_levels"],
        score_levels: TensorType["batch_size", "n_visited_levels"],
        last_count_levels: TensorType["batch_size", "n_visited_levels"],
        last_episode: TensorType[1]
    ) -> TensorType[1]:
        """Sampling a level from the replay distribution."""
        # sample replay decision
        dist = Bernoulli(probs=0.5)
        
        # TODO: support batch
        mask = ~torch.isin(self.levels, visited_levels)
        unseen_levels = self.levels[mask]
    
        if dist.sample() == 0 and len(unseen_levels) > 0:            
            # sample an unseen level
            uniform_dist = torch.rand(len(unseen_levels))
            selected_index = torch.argmax(uniform_dist)
            next_level = unseen_levels[selected_index]
            
            idx_selected_unseen_level = torch.where(self.levels == next_level)[0][0]
            self.visited_count_levels[idx_selected_unseen_level] = 1
        else:
            # sample a level for replay
            prioritized_dist = self.prioritized_dist.create(
                score_levels,
                last_count_levels,
                last_episode
            )
            
            visited_idx = torch.multinomial(prioritized_dist, num_samples=1)
            next_level = visited_levels[visited_idx][0] # return the first batch
            
        return next_level